In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tabulate import tabulate
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tabulate import tabulate

In [ ]:
df = pd.read_csv("dataset.csv") ##I already extracted the .gz file and created a dataset from it

In [ ]:
df

,Review,Sentiment
0,Bromwell High is a cartoon comedy. It ran at t...,pos
1,Homelessness (or Houselessness as George Carli...,pos
2,Brilliant over-acting by Lesley Ann Warren. Be...,pos
3,This is easily the most underrated film inn th...,pos
4,This is not the typical Mel Brooks film. It wa...,pos
...,...,...
49995,I occasionally let my kids watch this garbage ...,neg
49996,When all we have anymore is pretty much realit...,neg
49997,The basic genre is a thriller intercut with an...,neg
49998,Four things intrigued me as to this film - fir...,neg


In [ ]:
df.head()

,Review,Sentiment
0,Bromwell High is a cartoon comedy. It ran at t...,pos
1,Homelessness (or Houselessness as George Carli...,pos
2,Brilliant over-acting by Lesley Ann Warren. Be...,pos
3,This is easily the most underrated film inn th...,pos
4,This is not the typical Mel Brooks film. It wa...,pos


In [ ]:
df.dtypes

Review       object
Sentiment    object
dtype: object

In [ ]:
df.Review

0        Bromwell High is a cartoon comedy. It ran at t...
1        Homelessness (or Houselessness as George Carli...
2        Brilliant over-acting by Lesley Ann Warren. Be...
3        This is easily the most underrated film inn th...
4        This is not the typical Mel Brooks film. It wa...
                               ...                        
49995    I occasionally let my kids watch this garbage ...
49996    When all we have anymore is pretty much realit...
49997    The basic genre is a thriller intercut with an...
49998    Four things intrigued me as to this film - fir...
49999    David Bryce's comments nearby are exceptionall...
Name: Review, Length: 50000, dtype: object

In [ ]:
import re

In [ ]:
def preprocess_text(text):
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = text.replace('unk', '')
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    return text

df['Review'] = df['Review'].apply(preprocess_text)
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == 'pos' else 0)
print(df)

                                                  Review  Sentiment
0      Bromwell High is a cartoon comedy It ran at th...          1
1      Homelessness or Houselessness as George Carlin...          1
2      Brilliant overacting by Lesley Ann Warren Best...          1
3      This is easily the most underrated film inn th...          1
4      This is not the typical Mel Brooks film It was...          1
...                                                  ...        ...
49995  I occasionally let my kids watch this garbage ...          0
49996  When all we have anymore is pretty much realit...          0
49997  The basic genre is a thriller intercut with an...          0
49998  Four things intrigued me as to this film  firs...          0
49999  David Bryces comments nearby are exceptionally...          0

[50000 rows x 2 columns]


In [ ]:
df['Review'] = df['Review'].str.lower()
print(df)

                                                  Review  Sentiment
0      bromwell high is a cartoon comedy it ran at th...          1
1      homelessness or houselessness as george carlin...          1
2      brilliant overacting by lesley ann warren best...          1
3      this is easily the most underrated film inn th...          1
4      this is not the typical mel brooks film it was...          1
...                                                  ...        ...
49995  i occasionally let my kids watch this garbage ...          0
49996  when all we have anymore is pretty much realit...          0
49997  the basic genre is a thriller intercut with an...          0
49998  four things intrigued me as to this film  firs...          0
49999  david bryces comments nearby are exceptionally...          0

[50000 rows x 2 columns]


In [ ]:
df.Review

0        bromwell high is a cartoon comedy it ran at th...
1        homelessness or houselessness as george carlin...
2        brilliant overacting by lesley ann warren best...
3        this is easily the most underrated film inn th...
4        this is not the typical mel brooks film it was...
                               ...                        
49995    i occasionally let my kids watch this garbage ...
49996    when all we have anymore is pretty much realit...
49997    the basic genre is a thriller intercut with an...
49998    four things intrigued me as to this film  firs...
49999    david bryces comments nearby are exceptionally...
Name: Review, Length: 50000, dtype: object

In [ ]:
X = df['Review']
y = df["Sentiment"]

In [ ]:
!pip install tabulate

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=42)  #tried with full dataset(colab crashed)

In [ ]:
vectorizer = CountVectorizer()

In [ ]:
X_train_bow = vectorizer.fit_transform(X_train)

In [ ]:
X_test_bow = vectorizer.transform(X_test)

In [ ]:
max_rows = 30000  ##IT WAS CRASHING FOR MORE THAN THIS
if X.shape[0] > max_rows:
    # Randomly select max_rows number of rows
    indices = np.random.choice(X.shape[0], max_rows, replace=False)

    # Subset the dataset based on the selected indices
    X_subset = X[indices]
    y_subset = y[indices]
else:
    # If the dataset is already within the memory limits, use the original dataset
    X_subset = X
    y_subset = y

# Split the subset dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=0.25, random_state=42) #working on subset

In [ ]:
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)
X_train_bow = torch.Tensor(X_train_bow.toarray())
X_test_bow = torch.Tensor(X_test_bow.toarray())
y_train = torch.Tensor(y_train.values)  # Convert y_train to a NumPy array
y_test = torch.Tensor(y_test.values)  # Convert y_test to a NumPy array

In [ ]:
# Create DataLoader for batch processing
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
class SequentialModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model_type):
        super(SequentialModel, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.model_type = model_type

        if model_type == 'RNN':
            self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        elif model_type == 'GRU':
            self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True)
        elif model_type == 'LSTM':
            self.rnn = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        elif model_type == 'BiLSTM':
            self.rnn = nn.LSTM(hidden_size, hidden_size, batch_first=True, bidirectional=True)

        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)

        if self.model_type == 'RNN' or self.model_type == 'GRU':
            output, _ = self.rnn(embedded)
        else:
            output, (_, _) = self.rnn(embedded)

        if self.model_type == 'BiLSTM':
            output = output[:, -1, :]

        output = self.fc(output[:, -1, :])
        return output

In [ ]:
#parameters for the models
input_size = len(vectorizer.vocabulary_)
hidden_size = 100
output_size = 1

In [ ]:
models = {
    'RNN': SequentialModel(input_size, hidden_size, output_size, model_type='RNN'),
    'GRU': SequentialModel(input_size, hidden_size, output_size, model_type='GRU'),
    'LSTM': SequentialModel(input_size, hidden_size, output_size, model_type='LSTM'),
    'BiLSTM': SequentialModel(input_size, hidden_size, output_size, model_type='BiLSTM')
}


In [ ]:
#loss function and optimizer
loss_function = nn.BCEWithLogitsLoss()
optimizers = {model_type: optim.Adam(models[model_type].parameters(), lr=0.001) for model_type in models}

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        for model_type in models:
            optimizers[model_type].zero_grad()
            output = models[model_type](batch_X.long())
            loss = loss_function(output.squeeze(), batch_y)
            loss.backward()
            optimizers[model_type].step()

In [ ]:
results = {}
with torch.no_grad():
    for model_type in models:
        models[model_type].eval()
        test_predictions = models[model_type](X_test_tensor.long())
        test_predictions = torch.sigmoid(test_predictions).squeeze().numpy()
        test_predictions = np.where(test_predictions >= 0.5, 1, 0)
        accuracy = accuracy_score(y_test, test_predictions)
        precision = precision_score(y_test, test_predictions)
        recall = recall_score(y_test, test_predictions)
        f1 = f1_score(y_test, test_predictions)

        results[model_type] = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1
        }

In [ ]:
table_data = []
for model_type in results:
    table_data.append([model_type] + list(results[model_type].values()))

table = tabulate(table_data, headers=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'], tablefmt='orgtbl')

print(table)

| Model   |   Accuracy |   Precision |   Recall |   F1-Score |
|---------+------------+-------------+----------+------------|
| RNN     |       0.85 |        0.82 |     0.87 |       0.84 |
| GRU     |       0.86 |        0.84 |     0.87 |       0.85 |
| LSTM    |       0.87 |        0.85 |     0.88 |       0.86 |
| BiLSTM  |       0.88 |        0.86 |     0.89 |       0.87 |
